In [1]:
import os
os.environ["MY_VAR"]

KeyError: 'MY_VAR'

# FDTD Meep

[Meep](https://meep.readthedocs.io/en/latest/) is a free, open source Finite Difference Time Domain (FDTD) simulator

You to install meep and MPB with:


conda

```
conda install -c conda-forge pymeep=*=mpi_mpich_* -y
```

or mamba (faster conda)

```
mamba install pymeep=*=mpi_mpich_* -y
```

Works only on Mac, Linux or windows [WSL](https://docs.microsoft.com/en-us/windows/wsl/)



gdsfactory `gmeep` plugin computes the transmission spectrum for Photonic planar components.

One of the advantages of using `gmeep` is that you only need to define your component once using gdsfactory, and automatically can simulate it in meep without having to define the geometry again.

For extracting Sparameters, `gmeep` automatically swaps the source between ports to compute the full Sparameters matrix.

- add monitors on each component port
- extend ports to go over the PML
- run simulation and compute Sparameter coefficients with proper ratios for each Sparameter. Monitors record Fourier Transform fields. Sparameter is a relationship of those parameters at different frequencies.

The resolution is in pixels/um, you should run with at least `resolution=30` for 1/30 um/pixel (33 nm/ pixel)

Notice that most examples run with `resolution=20` so they run fast.

Here are some examples on how to extract Sparameters in Meep in some planar devices.


```bash

         top view
              ________________________________
             |                               |
             | xmargin_left                  | port_extension
             |<--------->       port_margin ||<-->
          o2_|___________          _________||_o3
             |           \        /          |
             |            \      /           |
             |             ======            |
             |            /      \           |
          o1_|___________/        \__________|_o4
             |   |                 <-------->|
             |   |ymargin_bot   xmargin_right|
             |   |                           |
             |___|___________________________|

        side view
              ________________________________
             |                     |         |
             |                     |         |
             |                   zmargin_top |
             |xmargin_left         |         |
             |<---> _____         _|___      |
             |     |     |       |     |     |
             |     |     |       |     |     |
             |     |_____|       |_____|     |
             |       |                       |
             |       |                       |
             |       |zmargin_bot            |
             |       |                       |
             |_______|_______________________|



```

## Single core

Running on a single CPU core can be slow as the a single core needs to update all the simulation grid points sequentially.

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import gdsfactory as gf
import gdsfactory.simulation.gmeep as gm

gf.config.set_plot_options(show_subports=False, show_ports=False)

2022-04-09 17:31:31.543 | INFO     | gdsfactory.config:<module>:52 - Load '/home/tzhuang/Downloads/gdsfactory/gdsfactory' 4.4.14


ModuleNotFoundError: No module named 'meep.materials'

In [2]:
import sys
print(sys.path)
#sys.path.append('C:\\Users\\Timmy\\gdsfactory')
sys.path.append('/home/tzhuang/Downloads/gdsfactory')

['/home/tzhuang/Downloads/gdsfactory/docs/notebooks/plugins/meep', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/tzhuang/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']


In [ ]:
c = gf.components.straight(length=2)
c

`run=False` only plots the simulations for you to review that is set up **correctly**

In [ ]:
df = gm.write_sparameters_meep(c, run=False, ymargin_top=3, ymargin_bot=3)

In [ ]:
help(gm.write_sparameters_meep)

As you've noticed we added `ymargin_top` and `ymargin_bot` to ensure we have enough distance to the PML

You can also do this directly with another version of the function that adds `ymargin_top` and `ymargin_bot`

In [ ]:
c = gf.components.straight(length=2)
df = gm.write_sparameters_meep(c, run=False)

Because components with `left-right` ports are very common `write_sparameters_meep` `y_margin = 3um `

In [ ]:
c = gf.components.taper(length=2.0, width1=0.5, width2=1, with_cladding_box=True)
c

In [ ]:
df = gm.write_sparameters_meep(c, run=False)

In [ ]:
df = gm.write_sparameters_meep(c, resolution=20)

In [ ]:
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=("s21m",), logscale=False)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=("s21m",))

For a small taper S21 (Transmission) is around 0dB (100% transmission)

## Port symmetries

You can save some simulations in reciprocal devices.
If the device looks the same going from in -> out as out -> in, we only need to run one simulation

In [ ]:
c = gf.components.bend_euler(radius=3)
c

In [ ]:
df = gm.write_sparameters_meep_1x1_bend90(c, run=False)

In [ ]:
df = gm.write_sparameters_meep_1x1_bend90(c, run=True)

In [ ]:
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=("s21m",), logscale=False)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=("s11m",))

In [ ]:
c = gf.components.crossing()
c

Here are the port symmetries for a crossing

```python
port_symmetries = {
    "o1": {
        "s11": ["s22", "s33", "s44"],
        "s21": ["s12", "s34", "s43"],
        "s31": ["s13", "s24", "s42"],
        "s41": ["s14", "s23", "s32"],
    }
}
```

In [ ]:
df = gm.write_sparameters_meep(
    c,
    resolution=20,
    ymargin=0,
    port_symmetries=gm.port_symmetries.port_symmetries_crossing,
    run=False,
)

In [ ]:
df = gm.write_sparameters_meep(
    c,
    resolution=20,
    ymargin=0,
    port_symmetries=gm.port_symmetries.port_symmetries_crossing,
    run=True,
)

In [ ]:
gm.plot.plot_sparameters(df)

In [ ]:
gm.plot.plot_sparameters(df, keys=("s31m",))

As you can see this crossing looks beautiful but is quite **lossy** (9dB @ 15550nm)

## Multicore (MPI)

You can divide each simulation into multiple cores thanks to [MPI (message passing interface)](https://en.wikipedia.org/wiki/Message_Passing_Interface)

Lets try to reproduce the coupler results from the [Meep docs](https://meep.readthedocs.io/en/latest/Python_Tutorials/GDSII_Import/)

According to the simulations in the doc to get a 3dB (50%/50%) splitter you need 150nm over 8um

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

import gdsfactory as gf
import gdsfactory.simulation as sim
import gdsfactory.simulation.gmeep as gm

In [ ]:
help(gf.components.coupler)

In [ ]:
c = gf.components.coupler(length=8, gap=0.13)
c

In [ ]:
gm.write_sparameters_meep(component=c, run=False)

In [ ]:
filepath = gm.write_sparameters_meep_mpi(
    component=c,
    cores=4,
    resolution=30,
)

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=["s13m", "s14m"])

## Batch

You can also run a batch of multicore simulations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gdsfactory as gf

import gdsfactory.simulation as sim
import gdsfactory.simulation.gmeep as gm

In [ ]:
c = gf.components.straight(length=3.1)

In [ ]:
gm.write_sparameters_meep(c, ymargin=3, run=False)

In [ ]:
c1_dict = {"component": c, "ymargin": 3}
jobs = [
    c1_dict,
]

filepaths = gm.write_sparameters_meep_batch_1x1(
    jobs=jobs,
    cores_per_run=4,
    total_cores=8,
    lazy_parallelism=True,
)

In [ ]:
df = pd.read_csv(filepaths[0])
gf.simulation.plot.plot_sparameters(df)

In [ ]:
c = gf.components.coupler_ring()
c

In [ ]:
p = 2.5
gm.write_sparameters_meep(c, ymargin=0, ymargin_bot=p, xmargin=p, run=False)

In [ ]:
c1_dict = dict(
    component=c,
    ymargin=0,
    ymargin_bot=p,
    xmargin=p,
)
jobs = [c1_dict]

filepaths = gm.write_sparameters_meep_batch(
    jobs=jobs,
    cores_per_run=4,
    total_cores=8,
    delete_temp_files=False,
    lazy_parallelism=True,
)

In [ ]:
df = pd.read_csv(filepaths[0])

In [ ]:
gm.plot.plot_sparameters(df)

In [ ]:
gm.plot.plot_sparameters(df, keys=["s31m", "s41m"])

In [ ]:
gm.plot.plot_sparameters(df, keys=["s31m"])

In [ ]:
gm.plot.plot_sparameters(df, keys=["s41m"])